<a href="https://colab.research.google.com/github/sophianicogos/VigenereCOLOR/blob/main/toolboxHCP_retinotopy_averageSUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ccnmaastricht/CNI_toolbox.git


Cloning into 'CNI_toolbox'...
remote: Enumerating objects: 1411, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 1411 (delta 150), reused 267 (delta 141), pack-reused 1118
Receiving objects: 100% (1411/1411), 943.45 KiB | 10.25 MiB/s, done.
Resolving deltas: 100% (710/710), done.


### ♦ PACKAGE SETUP ♦

In [2]:
import nibabel as nib
import numpy as np

In [51]:
import zipfile
import os

# Path to the uploaded ZIP file
zip_file_path = '/cni_tlbx.zip'  # Adjust the path if necessary

# Destination directory for extraction
extraction_path = '/'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# List the contents of the extraction directory
os.listdir(extraction_path)


import os
os.chdir('/')
!pwd
from cni_tlbx import IRM


/


### DATA HANDLING

In [4]:
NIFTI = nib.load('/content/GlasserParcellation_niftiConvert.nii')
header = NIFTI.header
affine = NIFTI.affine
data = np.array(NIFTI.dataobj)
data = np.transpose(data, (3, 0, 1, 2))

### pRF MAPPING

instantiation?

Input referred models?

In [59]:
from IRM import IRM
data = data  # Assuming you have your functional MRI data tensor
sampling_frequency = 1  # Sampling frequency of data acquisition

# Extracting dimensions of the data tensor
n_samples, n_rows, n_cols, n_slices = data.shape

# Creating a dictionary of parameters
parameters = {
    'f_sampling': sampling_frequency,
    'n_samples': n_samples,
    'n_rows': n_rows,
    'n_cols': n_cols,
    'n_slices': n_slices
}

# Instantiating the IRM tool with the parameters
irm = IRM(parameters)


usage..

In [55]:
import os

# Set the current directory to be 'cni_tlbx'
os.chdir('cni_tlbx')

In [57]:
stimulus_file_path = "/7T_RETEXP.npy"
irm.set_stimulus(stimulus_file_path)

In [58]:
irm.create_timecourse(f,xdata)

AttributeError: 'IRM' object has no attribute 'create_timecourse'

In [ ]:
results = irm.mapping(data)

pRF Mapping maybe just do it?

In [98]:
import os
import numpy as np
from PIL import Image
import io

from google.colab import files

print("Please upload image files (PNG format):")
uploaded_files = files.upload()

processed_images = []
for image_name, image_data in uploaded_files.items():
    # Read the image from the uploaded file object
    image = Image.open(io.BytesIO(image_data))

    h_stimulus, w_stimulus = image.size
    # greyscale
    image = image.convert('L')
    # Convert the image to a NumPy array and append it to the list
    processed_images.append(np.array(image))

# Stack the processed images along a new dimension to create a 3D tensor
stimulus_tensor = np.stack(processed_images, axis=0)



Please upload image files (PNG format):


Saving 2nov.jpeg to 2nov (3).jpeg


In [99]:
if data.ndim==4:
    n_samples, n_rows, n_cols, n_slices = data.shape
elif data.ndim==2:
    n_samples, n_rows = data.shape
    n_cols, n_slices = 1, 1

parameters = {'f_sampling': sampling_frequency,
        'h_stimulus': h_stimulus,
        'w_stimulus': w_stimulus,
        'n_samples': n_samples,
        'n_rows': n_rows,
        'n_cols': n_cols,
        'n_slices': n_slices}

from cni_tlbx import pRF

# Instantiate the PRF tool
prf = pRF(parameters)


Reshape stimulus_tensor to work

In [102]:
import numpy as np

# Print the shape of stimulus_tensor
print("Shape of stimulus_tensor:", stimulus_tensor.shape)

# Verify that the number of samples matches n_samples
n_samples_expected = parameters['n_samples']
n_samples_actual = stimulus_tensor.shape[-1]  # Get the number of samples from the last dimension
print("Number of samples expected:", n_samples_expected)
print("Number of samples actual:", n_samples_actual)

# Reshape stimulus_tensor if needed
if stimulus_tensor.ndim == 4:  # Ensure it has four dimensions
    stimulus_tensor = np.squeeze(stimulus_tensor, axis=(2, 3))  # Remove extra dimensions for singleton dimensions

# Reshape stimulus_tensor to match expected shape
if stimulus_tensor.shape[0] != parameters['h_stimulus'] or stimulus_tensor.shape[1] != parameters['w_stimulus']:
    stimulus_tensor = np.reshape(stimulus_tensor,
                                 (parameters['h_stimulus'], parameters['w_stimulus'], n_samples_expected))
    print("Reshaped stimulus_tensor to match expected.")





Shape of stimulus_tensor: (347, 347, 1)
Number of samples expected: 1
Number of samples actual: 1


In [103]:
prf.set_stimulus(stimulus_tensor)

Create timecourse

In [105]:
prf.create_timecourses()
# use_slope = true,             # flag to indicate whether to explore slopes (true) or sigmas directly (false)
                      #  num_xy = number_xy,           # number of points in x and y dimension
                      #  num_size = number_sizes,      # number of points in the size dimension (either slopes or sigmas)
                      #  max_radius = max_radius,      # maximum radius of stimulated visual field
                      #  min_slope = min_slope,        # lower bound of slope (only needed when use_slope = true)
                      #  max_slope = max_slope,        # upper bound of slope (only needed when use_slope = true)
                      #  min_sigma = min_sigma,        # lower bound of sigma (only needed when use_slope = false)
                      #  max_sigma = max_sigma,        # upper bound of sigma (only needed when use_slope = False)
                      #  css_exponent = alpha,         # exponent for compressive spatial summation
                      #  sampling = 'log',             # location sampling type ('log' or 'linear')


creating timecourses
[====================] 100%

### Saving resuts (future)

In [ ]:
header["dim"][0] = 3
header["dim"][4] = 1

for key, value in results.items():
    results_nii = nib.Nifti1Image(value, affine, header)
    nib.save(results_nii, "%s.nii.gz" % key)


can i combine these?